In [8]:
import torch
import numpy as np
from sentence_transformers import SentenceTransformer
from converter.converter import sentence_transformers_onnx

In [2]:
# TODO: ONNX not working on GPU
device = torch.device("cpu" if torch.cuda.is_available() else "cpu")

In [3]:
# load the model
model = SentenceTransformer.load('./results/domain_adaptation_model')

In [4]:
# Transform the model to use onnx format
onnx_model = sentence_transformers_onnx(
    model,
    output_path="triton/model_repository/domain_adaptation_model/1/model",
    config_path="results/domain_adaptation_model",
    device=device
)
onnx_model

In [14]:
# Compute embeddings for two textual contents and compute dot product
green_embedding = onnx_model.encode("Composable Lightweight Processors")
tree_embedding = onnx_model.encode("ocean")

# Same results as Pytorch-based model - so conversion seems accurate
np.dot(green_embedding, tree_embedding)

hola
['CPUExecutionProvider']
hola
['CPUExecutionProvider']


-0.1352806

In [20]:
import onnx
model = onnx.load(r"triton/model_repository/domain_adaptation_model/1/model.onx")

# model is an onnx model
graph = model.graph
# graph inputs
for input_name in graph.input:
    print(input_name)
# graph outputs
for output_name in graph.output:
    print(output_name)
# iterate over nodes
for node in graph.node:
    # node inputs
    for idx, node_input_name in enumerate(node.input):
        print(idx, node_input_name)
    # node outputs
    for idx, node_output_name in enumerate(node.output):
        print(idx, node_output_name)

name: "input_ids"
type {
  tensor_type {
    elem_type: 7
    shape {
      dim {
        dim_param: "batch_size"
      }
      dim {
        dim_param: "max_seq_len"
      }
    }
  }
}

name: "attention_mask"
type {
  tensor_type {
    elem_type: 7
    shape {
      dim {
        dim_param: "batch_size"
      }
      dim {
        dim_param: "max_seq_len"
      }
    }
  }
}

name: "token_type_ids"
type {
  tensor_type {
    elem_type: 7
    shape {
      dim {
        dim_param: "batch_size"
      }
      dim {
        dim_param: "max_seq_len"
      }
    }
  }
}

name: "start"
type {
  tensor_type {
    elem_type: 1
    shape {
      dim {
        dim_param: "batch_size"
      }
      dim {
        dim_param: "max_seq_len"
      }
      dim {
        dim_value: 384
      }
    }
  }
}

name: "end"
type {
  tensor_type {
    elem_type: 1
    shape {
      dim {
        dim_param: "batch_size"
      }
      dim {
        dim_param: "max_seq_len"
      }
    }
  }
}

0 input_ids
0 /Sh

In [11]:
import numpy as np
from torchvision import transforms
from PIL import Image
import tritonclient.http as httpclient
from tritonclient.utils import triton_to_np_dtype

# Setting up client
client = httpclient.InferenceServerClient(url="localhost:8000")

inputs = httpclient.InferInput("input__0", (3,), datatype="FP32")
inputs.set_data_from_numpy(np.array([1,2,3], dtype='float32'), binary_data=True)

outputs = httpclient.InferRequestedOutput("output__0", binary_data=True, class_count=1000)

# Querying the server
results = client.infer(model_name="results/domain_adaptation_model/", inputs=[inputs], outputs=[outputs])
inference_output = results.as_numpy('start')
print(inference_output[:5])

InferenceServerException: [404] Not Found